In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.ticker import MaxNLocator

In [ ]:
# EPSvsNt1 --> Nh=100 
# EPSvsNt2 --> Nh=200
# EPSvsNt3 --> Nh=50
# EPSvsNt4 --> Nh=150

Nh = 100

path = "/home/riccardo/Desktop/PhD/MyRepos/FD-PyIMEX-RB/__RESULTS/AdvDiff2D/EPSvsNt1"
results = np.load(os.path.join(path, "results.npz"), allow_pickle=True)

In [ ]:
Nt_FE = results['Nt_FE'][None][0][0]

eps = results['eps_values'][None][0]
Nts = results['Nt_values'][None][0]

errIMEX = results['errors_l2'][None][0]['IMEX-RB']
errBE = results['errors_l2'][None][0]['BE']
errFE = results['errors_l2'][None][0]['FE']

timesIMEX = results['times'][None][0]['IMEX-RB']
timesBE = results['times'][None][0]['BE']
timesFE = results['times'][None][0]['FE']

subiters = results['subiters'][None][0]['IMEX-RB']
mask = np.zeros_like(subiters, dtype=bool)
for i, _Nt in enumerate(Nts):
    mask[i, :, :_Nt] = True
masked_subiters = np.ma.masked_array(subiters, mask=~mask)

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

fig, ax = plt.subplots(figsize=(6,6))

ax.loglog(1 / Nts, errBE, '-s', markersize=8, color="green", label="BE")
# ax.loglog(1 / Nts, errFE, '-s', markersize=8, color="blue", label="FE")

ax.loglog(1 / Nts, errIMEX[:, 0], '-v', markersize=6, color="purple", 
           label=r"IMEX-RB - $\varepsilon = 100 \ \bar{\varepsilon}$")
ax.loglog(1 / Nts, errIMEX[:, 1], '-*', markersize=6, color="blueviolet", 
           label=r"IMEX-RB - $\varepsilon = 50 \ \bar{\varepsilon}$")
ax.loglog(1 / Nts, errIMEX[:, 2], '-X', markersize=6, color="magenta", 
           label=r"IMEX-RB - $\varepsilon = 10 \ \bar{\varepsilon}$")
ax.loglog(1 / Nts, errIMEX[:, 3], '-d', markersize=6, color="hotpink", 
           label=r"IMEX-RB - $\varepsilon = 5  \ \bar{\varepsilon}$")
ax.loglog(1 / Nts, errIMEX[:, 4], '-o', markersize=6, color="red", 
           label=r"IMEX-RB - $\varepsilon = \bar{\varepsilon}$")
#ax.loglog(1 / Nts, errIMEX[:, 6], '-o', markersize=6, color="maroon", 
#           label=r"IMEX-RB - $\varepsilon = 0.1 \ \bar{\varepsilon}$")

ax.loglog(1 / Nts, (Nts[0] * errBE[0] * 0.9) / Nts, 
          color="k", linestyle='--', linewidth=1)

x0, y0 = 0.0015, 0.0015
dx = 0.001
dy = dx  # for slope 1: dy = dx
ax.plot([x0, x0+dx], [y0, y0], 'k-', linewidth=.75)        
ax.plot([x0+dx, x0+dx], [y0, y0+dy], 'k-', linewidth=.75)        
ax.plot([x0+dx, x0], [y0+dy, y0], 'k-', linewidth=.75)     
ax.text(x0, y0-dy/2.5, r"$\mathcal{O}(\Delta t)$", fontsize=15, ha='left', va='bottom')

ax.axvline(1 / Nt_FE, color="k", linestyle='-.') 
ax.text(1.3e-3, 1.2e-2, r"$\Delta t_{FE}$", fontsize=15, color='k')

# ax.set_aspect('equal')

# Create an inset axis inside the main axis
axins = inset_axes(ax, width="45%", height="35%", loc='upper left')
axins.loglog(1 / Nts, errBE, '-s', markersize=8, color="green", label="BE")
axins.loglog(1 / Nts, errIMEX[:, 0], '-v', markersize=6, color="purple", 
           label=r"IMEX-RB - $\varepsilon = 100 \ \bar{\varepsilon}$")
axins.loglog(1 / Nts, errIMEX[:, 1], '-*', markersize=6, color="blueviolet", 
           label=r"IMEX-RB - $\varepsilon = 50 \ \bar{\varepsilon}$")
axins.loglog(1 / Nts, errIMEX[:, 2], '-X', markersize=6, color="magenta", 
           label=r"IMEX-RB - $\varepsilon = 10 \ \bar{\varepsilon}$")
axins.loglog(1 / Nts, errIMEX[:, 3], '-d', markersize=6, color="hotpink", 
           label=r"IMEX-RB - $\varepsilon = 5  \ \bar{\varepsilon}$")
axins.loglog(1 / Nts, errIMEX[:, 4], '-o', markersize=6, color="red", 
           label=r"IMEX-RB - $\varepsilon = \bar{\varepsilon}$")
#axins.loglog(1 / Nts, errIMEX[:, 6], '-o', markersize=6, color="maroon", 
#             label=r"IMEX-RB - $\varepsilon = 0.1 \ \bar{\varepsilon}$")
axins.grid(which="major", linewidth=1, linestyle='--')

# Set the zoomed-in region limits
axins.set_xlim(1.4e-2, 1.75e-2)
axins.set_ylim(2.25e-2, 2.9e-2)

# Turn off ticks and numbers cleanly
axins.tick_params(
    axis='both',        # both axes
    which='both',       # both major and minor ticks
    bottom=False,       # no bottom ticks
    top=False,          # no top ticks
    left=False,         # no left ticks
    right=False,        # no right ticks
    labelbottom=False,  # no x-axis labels
    labelleft=False     # no y-axis labels
)

# Draw lines connecting the inset region to the main plot
mark_inset(ax, axins, loc1=1, loc2=3, fc="none", ec="0.5")

ax.grid(which="major", linewidth=1, linestyle='--')
ax.grid(which="minor", linewidth=0.5, linestyle='--')
ax.tick_params(axis='both', which='major', labelsize=15)
ax.legend(fontsize=14, loc="lower right")

# ax.set_ylim([9e-4, 2.2e-1])

ax.set_xlabel(r"$\Delta t$", fontsize=17)
ax.set_ylabel("Relative error", fontsize=15)

plt.tight_layout()

plt.savefig(f'errors_eps_Nh_{Nh}.png', dpi=400)

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
ax.semilogx(1 / Nts, np.mean(masked_subiters[:, 0] + 1, axis=-1), '-v', markersize=6, color="purple", 
           label=r"$\varepsilon = 100 \ \bar{\varepsilon}$")
ax.semilogx(1 / Nts, np.mean(masked_subiters[:, 1] + 1, axis=-1), '-*', markersize=6, color="blueviolet", 
           label=r"$\varepsilon = 50 \ \bar{\varepsilon}$")
ax.semilogx(1 / Nts, np.mean(masked_subiters[:, 2] + 1, axis=-1), '-X', markersize=6, color="magenta", 
           label=r"$\varepsilon = 10 \ \bar{\varepsilon}$")
ax.semilogx(1 / Nts, np.mean(masked_subiters[:, 3] + 1, axis=-1), '-D', markersize=6, color="hotpink", 
           label=r"$\varepsilon = 5 \ \bar{\varepsilon}$")
ax.semilogx(1 / Nts, np.mean(masked_subiters[:, 4] + 1, axis=-1), '-o', markersize=6, color="red", 
           label=r"$\varepsilon = \bar{\varepsilon}$")
ax.semilogx(1 / Nts, np.mean(masked_subiters[:, 6] + 1, axis=-1), '-p', markersize=6, color="maroon", 
           label=r"$\varepsilon = 0.1 \ \bar{\varepsilon}$")

ax.axvline(1 / Nt_FE, color="k", linestyle='-.') 
ax.text(5.7e-3, 16.3, r"$\Delta t_{FE}$", fontsize=15, color='k')

ax.legend(fontsize=14)
ax.grid(which="major", linestyle='--', linewidth=1)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

ax.set_xlabel(r"$\Delta t$", fontsize=17)
ax.set_ylabel("Average Inner iterations", fontsize=15)

plt.savefig(f'subiters_eps_Nh_{Nh}.png', dpi=400)

In [ ]:
Nt_idx = 1  # Nt = 32

tvec = np.linspace(0,1, Nts[Nt_idx]+1)[1:]

fig, ax = plt.subplots(figsize=(6,6))
ax.plot(tvec, subiters[Nt_idx, 0, :Nts[Nt_idx]] + 1, '-v', markersize=3, color="purple", 
           label=r"$\varepsilon = 100 \ \bar{\varepsilon}$")
ax.plot(tvec, subiters[Nt_idx, 1, :Nts[Nt_idx]] + 1, '-*', markersize=3, color="blueviolet", 
           label=r"$\varepsilon = 50 \ \bar{\varepsilon}$")
ax.plot(tvec, subiters[Nt_idx, 2, :Nts[Nt_idx]] + 1, '-X', markersize=3, color="magenta", 
           label=r"$\varepsilon = 10 \ \bar{\varepsilon}$")
ax.plot(tvec, subiters[Nt_idx, 3, :Nts[Nt_idx]] + 1, '-D', markersize=3, color="hotpink", 
           label=r"$\varepsilon = 5 \ \bar{\varepsilon}$")
ax.plot(tvec, subiters[Nt_idx, 4, :Nts[Nt_idx]] + 1, '-o', markersize=3, color="red", 
           label=r"$\varepsilon = \bar{\varepsilon}$")
ax.plot(tvec, subiters[Nt_idx, 6, :Nts[Nt_idx]] + 1, '-p', markersize=3, color="maroon", 
           label=r"$\varepsilon = 0.1 \ \bar{\varepsilon}$")

ax.legend(fontsize=14)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.grid(which="major", linestyle='--', linewidth=1)

ax.set_xlim([0, 1+1/Nts[Nt_idx]])
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

ax.set_xlabel(r"$t$", fontsize=15)
ax.set_ylabel("Inner iterations", fontsize=15)

plt.savefig(f'subiters_time_eps_Nh_{Nh}.png', dpi=400)

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
ax.loglog(1 / Nts, timesBE, '--s', markersize=4, color="green", 
           label="BE")
#ax.loglog(1 / Nts[-3:], timesFE[-3:], '--o', markersize=4, color="blue", 
#           label="FE")

ax.loglog(1 / Nts, timesIMEX[:, 0], '-v', markersize=6, color="purple", 
           label=r"IMEX-RB - $\varepsilon = 100 \ \bar{\varepsilon}$")
ax.loglog(1 / Nts, timesIMEX[:, 1], '-*', markersize=6, color="blueviolet", 
           label=r"IMEX-RB - $\varepsilon = 50 \ \bar{\varepsilon}$")
ax.loglog(1 / Nts, timesIMEX[:, 2], '-X', markersize=6, color="magenta", 
           label=r"IMEX-RB - $\varepsilon = 10 \ \bar{\varepsilon}$")
ax.loglog(1 / Nts, timesIMEX[:, 3], '-D', markersize=6, color="hotpink", 
           label=r"IMEX-RB - $\varepsilon = 5 \ \bar{\varepsilon}$")
ax.loglog(1 / Nts, timesIMEX[:, 4], '-o', markersize=6, color="red", 
           label=r"IMEX-RB - $\varepsilon = \bar{\varepsilon}$")
ax.loglog(1 / Nts, timesIMEX[:, 6], '-p', markersize=6, color="maroon", 
           label=r"IMEX-RB - $\varepsilon = 0.1 \ \bar{\varepsilon}$")

ax.axvline(1 / Nt_FE, color="k", linestyle='-.') 
ax.text(1.35e-3, 1.1, r"$\Delta t_{FE}$", fontsize=15, color='k')

ax.legend(fontsize=14, loc="upper left")
ax.tick_params(axis='both', which='major', labelsize=15)
ax.grid(which="major", linestyle='--', linewidth=1)
ax.grid(which="minor", linestyle='--', linewidth=.5)

ax.set_xlabel(r"$\Delta t$", fontsize=17)
ax.set_ylabel("CPU time [s]", fontsize=15)

ax.set_ylim([None, 4e2])

plt.tight_layout()

plt.savefig(f'times_eps_Nh_{Nh}.png', dpi=400)